[![Notebook Tutorial](__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/en/tutorial/notebooks/water_intake_profile_calculator/)

# Select your IPTS 

<img src='__docs/__all/shift_enter.gif' />

In [1]:
from __code import system
from __code.water_intake_profile_calculator import WaterIntakeProfileCalculator, WaterIntakeProfileSelector

from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_water_intake_profile.ui')
from __code.roi_selection_ui import Interface

system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

pyuic5 /Users/j35/git/IPTS/python_notebooks/ui/ui_water_intake_profile.ui -o /Users/j35/git/IPTS/python_notebooks/__code/ui_water_intake_profile.py
** Using Debugging Mode! **


VBox(children=(HBox(children=(Label(value='IPTS-'), Text(value='', layout=Layout(width='10%')), Label(value='DOES NOT EXIST!', layout=Layout(width='20%'), _dom_classes=('result_label',)))), Label(value='OR'), HBox(children=(Label(value='Select Folder', layout=Layout(width='20%')), Select(layout=Layout(height='300px'), options=('BraggEdge', 'Brianne', 'DOE_letters', 'FITSimages', 'IPTS-12534', 'IPTS-13639_resonance_shawn', 'IPTS-14832', 'IPTS-15177', 'IPTS-16212', 'IPTS-16259', 'IPTS-16475', 'IPTS-16893', 'IPTS-17099', 'IPTS-17685', 'IPTS-17981', 'IPTS-18580', 'IPTS-18806', 'IPTS-18813', 'IPTS-19037', 'IPTS-19558', 'IPTS-19621', 'IPTS-19626', 'IPTS-20209-TOPAZ', 'IPTS_test_normalization', 'IPTS_testing_time_spectra', 'ImagesToRegisterForGian', 'Indu', 'Keita_code', 'Pour_ibeatles_video', 'ResonanceImaging', 'SENSITIVE', 'SarmaGUI', 'SphereCoordinates3D', 'TempData', 'gian', 'iBeatles_debugging', 'shared'), value='BraggEdge'))), Button(button_style='info', description='HELP', style=ButtonStyle())))

# Description 

This notebook will calculate the water intake profile vs time of a sample.

Here are the steps (**bold** for user input/manipulation)
 * **select the normalized images**
 * images sorted by time
 * **region of interest seleted**
 * profile of counts vs vertical-pixel calculated (**select integrated algorithm: mean/median/sum**)
 * water intake profile vs file index or vs time
 * **export profiles**

# Python Import 

In [2]:
%gui qt

In [3]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from ipywidgets.widgets import interact
import numpy as np
import os
import ipywe.fileselector
from ipywidgets import widgets
from IPython.core.display import display, HTML
import pytz
import datetime

from NeuNorm.normalization import Normalization

from __code.metadata_handler import MetadataHandler
from __code import file_handler


# Select Images to Process

In [4]:
o_water = WaterIntakeProfileCalculator(working_dir = system.System.get_working_dir())
o_water.select_data()

Button(button_style='info', description='HELP', style=ButtonStyle())

HTML(value='Please wait...')

VBox(children=(Label(value='Select Images ...', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-15177', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store                        |     Wed Feb 14 10:19:42 2018', ' Sample5_uptake_no bad images     |     Directory'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HTML(value='Please wait...')

VBox(children=(Label(value='Select Images ...', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-15177/Sample5_uptake_no bad images', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' Sample5_1min_r_0.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_1.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_10.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_11.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_12.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_13.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_14.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_15.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_16.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_17.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_18.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_19.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_2.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_20.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_21.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_22.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_23.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_24.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_25.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_26.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_27.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_28.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_29.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_3.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_30.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_31.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_32.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_33.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_34.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_35.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_36.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_37.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_38.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_39.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_4.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_40.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_41.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_42.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_43.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_44.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_45.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_46.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_47.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_48.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_49.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_5.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_50.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_51.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_52.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_53.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_54.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_55.tif     |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_6.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_7.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_8.tif      |     Sun Mar 13 08:37:12 2016', ' Sample5_1min_r_9.tif      |     Sun Mar

In [5]:
o_water.load()

HBox(children=(Label(value='Retrieving Time Stamp', layout=Layout(width='20%')), IntProgress(value=0, layout=Layout(width='50%'), max=13)))

HBox(children=(Label(value='Loading sample', layout=Layout(width='20%')), IntProgress(value=0, max=13)))

# Select Profile Region 

In [6]:
# #for debugging only
# import numpy as np
# data = []
# names = []
# time_stamp = []
# for i in range(10): 
#     data.append(np.random.random((500,500)))
#     names.append('file{}'.format(i))
#     time_stamp.append("{}".format(i))
# dict_data = {'list_data': data,
#             'list_images': names,
#             'list_time_stamp': time_stamp,
#             'list_time_stamp_user_format': time_stamp}
# o_gui = WaterIntakeProfileSelector(dict_data=dict_data)
# o_gui.show()

In [7]:
o_gui = WaterIntakeProfileSelector(dict_data=o_water.dict_files)
o_gui.show()

/Users/j35/git/IPTS/python_notebooks/__code/water_intake_profile_calculator.py:444: RuntimeWarning: invalid value encountered in greater
  _result_range = np.where(image > range_max)


# DEBUGGING FROM HERE 

In [7]:
import re
m=re.match(r"^(?P<part1>\w*)_(?P<index>\d+)$", "Sample5_1min_r_0")

In [9]:
m.group['index']

TypeError: 'builtin_function_or_method' object is not subscriptable

In [7]:
list_files= o_water.dict_files['list_images']
list_files

['/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-15177/Sample5_uptake_no bad images/Sample5_1min_r_0.tif',
 '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-15177/Sample5_uptake_no bad images/Sample5_1min_r_1.tif',
 '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-15177/Sample5_uptake_no bad images/Sample5_1min_r_10.tif',
 '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-15177/Sample5_uptake_no bad images/Sample5_1min_r_11.tif',
 '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-15177/Sample5_uptake_no bad images/Sample5_1min_r_12.tif',
 '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-15177/Sample5_uptake_no bad images/Sample5_1min_r_13.tif']

In [8]:
import re

In [11]:
formated_list_files = []

for _file in list_files:
    dirname = os.path.dirname(_file)
    basename = os.path.basename(_file)
    [base, ext] = os.path.splitext(basename)
    

NameError: name 'list_files' is not defined

In [13]:
file0 = "/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-15177/Sample5_uptake_no bad images/Sample5_1min_r_13.tif"
basename = os.path.basename(file0)
[base, ext] = os.path.splitext(basename)
m = re.match(r"^(?P<part1>\w*)_(?P<index>\d+)$", base)
print(m)

<_sre.SRE_Match object; span=(0, 17), match='Sample5_1min_r_13'>


In [17]:
m.group('index')

'13'